# Start here

In [1]:
import pandas as pd
import numpy as np
import random


exp_dat = pd.read_csv('trial_file_626708_20220524_092356_UTC.csv', header=0)
exp_dat.loc[exp_dat.condition1 == 'Distractor', 'type'] = 'distractor'
for dtarget, vnum in zip(exp_dat.loc[exp_dat.type == 'distractor'].target.unique(), range(1,13)):
    exp_dat.loc[exp_dat.target == dtarget, 'condition3'] = str(vnum)

In [2]:
trial_types = ['test', 'distractor']
vig_numbers = list(range(1, 13))
bel_types =  ['TB', 'FB', 'IG']
ascrip_types = ['Knows','Thinks']

stimuli = {trial_type:{vnum:{
                             'belief_manip':{b:None for b in bel_types},
                             'ascription':{a:None for a in ascrip_types}
                             }
                       for vnum in vig_numbers}
           for trial_type in trial_types }

In [3]:

for trial_type in trial_types:
    for vnum in vig_numbers:
        for bel in bel_types:
            for ascrip in ascrip_types:
                if trial_type == 'distractor':
                    condition = exp_dat.loc[(exp_dat.type == trial_type) &
                                            (exp_dat.condition3 == str(vnum)) &
                                            (exp_dat.subjectGroup == str(1))]
                else:
                    condition = exp_dat.loc[(exp_dat.type == trial_type) &
                                            (exp_dat.condition1 == ascrip) &
                                            (exp_dat.condition2 == bel) &
                                            (exp_dat.condition3 == str(vnum))]

                stimuli[trial_type][vnum]['belief_manip'][bel] = exp_dat.loc[condition.index.values[0] - 5: condition.index.values[0] - 2, 'title'].values.tolist()
                stimuli[trial_type][vnum]['ascription'][ascrip] = {'target':condition.target.values[0], 'crrct_answr':condition.key.values[0]}

# Add practice

In [4]:
# Adding Practice
p1 = ["Kevin is out grocery shopping at the local market. His wife told him to get sourdough bread and pasta for dinner that evening.",
      "At the market, Kevin picks up some pasta, but he cannot find any sourdough bread anywhere. He looks up and down every aisle and then finally asks someone who works at the market.",
      "It turns out that the bakery at the market no longer makes sourdough bread because it was never very popular. ",
      "Kevin goes home without bread of any kind. His wife is slightly annoyed."
      ]
p1_target = "Kevin asked someone at the store if they sold bread."
p2 = ["Sara's younger sister is late for dance class. Sara has a scooter that she could lend her sister for the afternoon.",
      "The scooter is brand-new and will get Sara's sister to her dance class right on time. But Sara wants to use her scooter to get to her friend's house for dinner later on.",
      "Sara doesn't think that her sister's dance class will be over in time for her to also be able to use the scooter to get to dinner.",
      "Sara doesn't lend the scooter to her sister. Her sister walks to dance class and is late."
      ]
p2_target = "Sara wanted to take her scooter to the park."
stimuli['practice'] = {1:{'belief_manip':{bel:p1 for bel in bel_types},
                       'ascription':{a:{'target':p1_target, 'crrct_answer': 'j'} for a in ascrip_types}
                    },
                       2:{'belief_manip':{bel:p2 for bel in bel_types},
                       'ascription':{a:{'target':p2_target, 'crrct_answer': 'f'} for a in ascrip_types}
                    }}



# save stim json

In [5]:
import json

with open('KB_stim.json', 'w') as outfile:
    outfile.write(json.dumps(stimuli))
#stimuli.to_json('KbeforeB_stim.js', orient='records')


# Randomizing Trials for a Single Participant

In [17]:
import json
import pandas as pd
import numpy as np
import random

trial_types = ['test', 'distractor']
vig_numbers = list(range(1, 13))
bel_types =  ['TB', 'FB', 'IG']
ascrip_types = ['Knows','Thinks']
# Opening JSON file
with open('KB_stim.json') as json_file:
    data = json.load(json_file)
'''
trial_num = db.Column(db.Integer, unique=False, index=True)
    practice_prompt = db.Column(db.VARCHAR(6))
    practice_correct = db.Column(db.Boolean)
    trial_type = db.Column(db.VARCHAR(10))
    scenario = db.Column(db.Integer)
    belief_type = db.Column(db.VARCHAR(2))  # True Belief (TB), False Belief (FB), No Belief/ignorance (IG)
    ascription_factive = db.Column(db.VARCHAR(10))  # "knows" or "thinks"
    target = db.Column(db.VARCHAR(400))
    target_onset = db.Column(db.DateTime)
    response_onset = db.Column(db.DateTime)
    response_key = db.Column(db.VARCHAR(1))
    sound_weird = db.Column(db.Integer)
    sound_normal = db.Column(db.Integer)
'''
s1 = pd.DataFrame({
    'trial_num': None,
    'prompt':None,
    'practice_correct': None,
    'trial_type':'test',
    'scenario':vig_numbers,
    'belief':None,
    'ascription':None,
    'target':None,
    'crrct_answer':None,
    'target_onset':None,
    'response_onset':None,
    'response_key':None,
    'sound weird':None,
    'sound_normal':None,
}).sample(frac=1)

In [18]:

s1['ascription'] = np.repeat(ascrip_types, len(vig_numbers)/len(ascrip_types))
s1 = s1.sample(frac=1)
s1['belief'] = np.repeat(bel_types, len(vig_numbers)/len(bel_types))
s2 = s1.copy()
s2['trial_type'] = 'distractor'
s3 = pd.concat([s1, s2], ignore_index=True).sample(frac=1).reset_index(drop=True)
s3['trial_num']= range(1, len(s3)+1)


In [19]:

n_practice = 8
s3a = pd.DataFrame({x:None for x in s3.columns.tolist()}, index=range(n_practice))
s3a.trial_type=['tf_practice','tf_practice','tf_practice','tf_practice','tf_practice','tf_practice', 'practice', 'practice']
s3a.trial_num = [999]* n_practice

tf_prac = ["False"] * 3 + ["True"] * 3
random.shuffle(tf_prac)
ca = ['f' if tf_p == 'False' else 'j' for tf_p in tf_prac]

for prac in ['1', '2']:
    tf_prac.append(data['practice'][prac]['belief_manip']['TB'])
s3a.prompt = tf_prac

pr_targ = [None] * 6 #data['practice'][prac]['ascription']['Knows']['target']
[pr_targ.append(data['practice'][prac]['ascription']['Knows']['target']) for prac in ['1', '2']]
s3a['target'] = pr_targ

[ca.append(data['practice'][prac]['ascription']['Knows']['crrct_answer']) for prac in ['1','2']]

s3a['crrct_answer'] = ca

In [20]:
s4 = s3a.append(s3, ignore_index=True)


In [21]:
for i, trl in s4.iterrows():
    break

In [22]:
trl

trial_num                   999
prompt                     True
practice_correct           None
trial_type          tf_practice
scenario                   None
belief                     None
ascription                 None
target                     None
crrct_answer                  j
target_onset               None
response_onset             None
response_key               None
sound weird                None
sound_normal               None
Name: 0, dtype: object

In [16]:
from KnowledgeBelief.models import Trial

#if trl.trial_type == 'tf_practice':
tt = Trial(trial_num=trl.trial_num,
           prompt=trl.prompt,
           trial_type=trl.trial_type,
           scenario=trl.scenario,
           belief_type=trl.belief,
           ascription_type=trl.ascription,
           target=trl.target,
           correct_answer=trl.crrct_answer
    )

trial_num                   999
practice_prompt           False
practice_correct           None
trial_type          tf_practice
scenario                   None
belief                     None
ascription                 None
target                     None
crrct_answer                  f
target_onset               None
response_onset             None
response_key               None
sound weird                None
sound_normal               None
Name: 0, dtype: object

In [35]:
from KnowledgeBelief.models import Trial

tt = Trial.query.filter_by(prolific_id='0.221752s485926526', trial_type='tf_practice', response_key=None ).first()
tt is None

True

In [22]:
random.sample(k=[['False'] * 5  + ['True']* 5],)

TypeError: sample() missing 1 required positional argument: 'population'

In [32]:

# Opening JSON file
with open('KB_stim.json') as json_file:
    data = json.load(json_file)

In [33]:
stimuli['test'][6]['belief_manip']['FB']

["Brent is five years old and colorblind. Like the majority of colorblind people, he can't tell the difference between red and green.",
 'Forgetting that Brent is colorblind, his aunt sends him to the grocery store to buy some red apples.',
 'Brent runs to the grocery store and asks the grocer for red apples. The grocer accidentally hands Brent green apples and Brent pays for them.',
 'When Brent gives his aunt the green apples, she remembers that he is colorblind and thanks him for going all the way to the store.']

In [35]:
data['test']['6']['belief_manip']['FB']

["Brent is five years old and colorblind. Like the majority of colorblind people, he can't tell the difference between red and green.",
 'Forgetting that Brent is colorblind, his aunt sends him to the grocery store to buy some red apples.',
 'Brent runs to the grocery store and asks the grocer for red apples. The grocer accidentally hands Brent green apples and Brent pays for them.',
 'When Brent gives his aunt the green apples, she remembers that he is colorblind and thanks him for going all the way to the store.']